# Load Data - CS 513 Data Cleaning Final Project

Team 162 - Miffy's Kitchen

In [2]:
import pandas as pd
from pathlib import Path

# Load raw CSVs into DataFrame
parent_folder = 'NYPL-menus'
dish = pd.read_csv(f"{parent_folder}/Dish.csv")
menu = pd.read_csv(f"{parent_folder}/Menu.csv")
menu_item = pd.read_csv(f"{parent_folder}/MenuItem.csv")
menu_page = pd.read_csv(f"{parent_folder}/MenuPage.csv")

# display(dish)
# display(menu)
# display(menu_item)
# display(menu_page)

# *Data Profiling*

In [3]:
dish.describe()

,id,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
count,423397.000000,0.0,423397.000000,423397.000000,423397.000000,423397.000000,394297.000000,394297.000000
mean,264456.594900,NaN,3.060489,3.146794,1675.514555,1679.299738,0.965265,1.603875
std,150489.070889,NaN,27.818178,29.962122,651.321461,651.934580,6.714564,12.696274
min,1.000000,NaN,0.000000,-6.000000,0.000000,0.000000,0.000000,0.000000
25%,132374.000000,NaN,1.000000,1.000000,1900.000000,1900.000000,0.000000,0.000000
50%,269636.000000,NaN,1.000000,1.000000,1914.000000,1917.000000,0.000000,0.000000
75%,397135.000000,NaN,1.000000,1.000000,1949.000000,1955.000000,0.400000,0.600000
max,515677.000000,NaN,7740.000000,8484.000000,2928.000000,2928.000000,1035.000000,3050.000000


In [5]:
df = dish
df.info(verbose=True)

columns_with_mv = df.columns[df.isnull().any()]
df[columns_with_mv].isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423397 entries, 0 to 423396
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              423397 non-null  int64  
 1   name            423397 non-null  object 
 2   description     0 non-null       float64
 3   menus_appeared  423397 non-null  int64  
 4   times_appeared  423397 non-null  int64  
 5   first_appeared  423397 non-null  int64  
 6   last_appeared   423397 non-null  int64  
 7   lowest_price    394297 non-null  float64
 8   highest_price   394297 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 29.1+ MB


description      423397
lowest_price      29100
highest_price     29100
dtype: int64

## MenuItem Profile

In [6]:
# null count as percentage or sum
fields = ["price", "high_price", "menu_page_id", "dish_id"]
null = menu_item[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
price            445916
high_price      1240821
menu_page_id          0
dish_id             241
dtype: int64
=== Percent ===
price           0.334589
high_price      0.931040
menu_page_id    0.000000
dish_id         0.000181
dtype: float64


In [7]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_item
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

pk_violations = bad_df.groupby(['id', 'id']).size().to_dict()
display(pk_violations)

# FK Verification
bad_fk_menu_item_to_dish = df[~df["dish_id"].isin(dish["id"])]
bad_fk_menu_item_to_menu_page = df[~df["menu_page_id"].isin(menu_page["id"])]
display(bad_fk_menu_item_to_dish)
display(bad_fk_menu_item_to_menu_page)

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


{}

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
17058,19171,6706,NaN,NaN,NaN,2011-04-21 15:37:27 UTC,2011-04-21 15:45:51 UTC,0.250000,0.910850
20044,22322,3817,NaN,NaN,NaN,2011-04-21 18:16:13 UTC,2011-04-21 18:17:01 UTC,0.341429,0.704721
28458,31566,7355,NaN,NaN,NaN,2011-04-22 02:50:52 UTC,2011-04-22 18:48:41 UTC,0.568571,0.346818
30291,33486,11348,NaN,NaN,NaN,2011-04-22 06:35:32 UTC,2011-04-22 06:39:39 UTC,0.164286,0.477882
33111,36645,7356,NaN,NaN,NaN,2011-04-22 16:23:28 UTC,2011-04-22 18:59:11 UTC,0.184286,0.346567
...,...,...,...,...,...,...,...,...,...
889158,932573,47175,NaN,NaN,NaN,2012-05-14 03:16:30 UTC,2012-05-14 03:22:36 UTC,0.687143,0.275966
930243,974746,62976,NaN,NaN,NaN,2012-06-20 05:17:58 UTC,2012-06-20 05:32:58 UTC,0.210000,0.115715
936817,981533,62740,NaN,NaN,NaN,2012-06-22 20:57:32 UTC,2012-06-22 20:58:18 UTC,0.175714,0.250947
939086,983976,65327,NaN,NaN,NaN,2012-06-23 21:47:35 UTC,2012-06-23 21:41:38 UTC,0.405714,0.176490


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


## MenuPage Profile

In [8]:
# null count as percentage or sum
fields = ["id", "menu_id"]
null = menu_page[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
id         0
menu_id    0
dtype: int64
=== Percent ===
id         0.0
menu_id    0.0
dtype: float64


In [10]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_page
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

pk_violations = bad_df.groupby(['id', 'id']).size().to_dict()
pk_violations

# FK Verification
bad_fk_menu_page = df[~df["menu_id"].isin(menu["id"])]
display(bad_fk_menu_page)

,id,menu_id,page_number,image_id,full_height,full_width,uuid


,id,menu_id,page_number,image_id,full_height,full_width,uuid


,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99
...,...,...,...,...,...,...,...
66922,77417,35525,2.0,5194206,2942.0,1990.0,af4751c0-6212-0132-6245-58d385a7bbd0
66923,77418,35525,3.0,5194207,2837.0,2628.0,af604d50-6212-0132-b41d-58d385a7bbd0
66924,77419,35525,4.0,5194208,2852.0,2002.0,af7910c0-6212-0132-e227-58d385a7bbd0
66925,77420,35525,5.0,5194209,2835.0,1990.0,af92d3f0-6212-0132-6b8d-58d385a7bbd0


# *Cleaning Verification*

In [4]:
parent_folder = 'NYPL-menus-clean'
# dish = pd.read_csv(f"{parent_folder}/Dish.csv")
# menu = pd.read_csv(f"{parent_folder}/Menu.csv")
menu_item_clean = pd.read_csv(f"{parent_folder}/MenuItem.csv")
menu_page_clean = pd.read_csv(f"{parent_folder}/MenuPage.csv")

## Dish Verify

In [17]:
# null count as percentage or sum
fields = ['description']
null = dish[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
description    423397
dtype: int64
=== Percent ===
description    1.0
dtype: float64


In [18]:
# IC Violations and Constraints

# check for duplicate rows
df = dish
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
counts = df.groupby('id')['name'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

ic3_violations = bad_df.groupby(['id', 'name']).size().to_dict()
ic3_violations

,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


{}

## MenuItem Verify

In [19]:
# null count as percentage or sum
fields = ["price", "high_price", "menu_page_id", "dish_id"]
null = menu_item_clean[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
price           445916
high_price           0
menu_page_id         0
dish_id            241
dtype: int64
=== Percent ===
price           0.334589
high_price      0.000000
menu_page_id    0.000000
dish_id         0.000181
dtype: float64


In [20]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_item_clean
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

pk_violations = bad_df.groupby(['id', 'id']).size().to_dict()
display(pk_violations)

# FK Verification
bad_fk_menu_item_to_dish = df[~df["dish_id"].isin(dish["id"])]
bad_fk_menu_item_to_menu_page = df[~df["menu_page_id"].isin(menu_page["id"])]
display(bad_fk_menu_item_to_dish)
display(bad_fk_menu_item_to_menu_page)

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


{}

,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos
17058,19171,6706,NaN,0.00,NaN,2011-04-21 15:37:27 UTC,2011-04-21 15:45:51 UTC,0.250000,0.910850
20044,22322,3817,NaN,0.00,NaN,2011-04-21 18:16:13 UTC,2011-04-21 18:17:01 UTC,0.341429,0.704721
28458,31566,7355,NaN,0.00,NaN,2011-04-22 02:50:52 UTC,2011-04-22 18:48:41 UTC,0.568571,0.346818
30291,33486,11348,NaN,0.00,NaN,2011-04-22 06:35:32 UTC,2011-04-22 06:39:39 UTC,0.164286,0.477882
33111,36645,7356,NaN,0.00,NaN,2011-04-22 16:23:28 UTC,2011-04-22 18:59:11 UTC,0.184286,0.346567
...,...,...,...,...,...,...,...,...,...
889158,932573,47175,NaN,0.00,NaN,2012-05-14 03:16:30 UTC,2012-05-14 03:22:36 UTC,0.687143,0.275966
930243,974746,62976,NaN,0.00,NaN,2012-06-20 05:17:58 UTC,2012-06-20 05:32:58 UTC,0.210000,0.115715
936817,981533,62740,NaN,0.00,NaN,2012-06-22 20:57:32 UTC,2012-06-22 20:58:18 UTC,0.175714,0.250947
939086,983976,65327,NaN,0.00,NaN,2012-06-23 21:47:35 UTC,2012-06-23 21:41:38 UTC,0.405714,0.176490


,id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


## MenuPage

In [21]:
# null count as percentage or sum
fields = ["id", "menu_id"]
null = menu_page_clean[fields].isnull()
counts = null.sum()

print("=== Count ===")
print(counts)
print("=== Percent ===")
print(null.mean())

=== Count ===
id         0
menu_id    0
dtype: int64
=== Percent ===
id         0.0
menu_id    0.0
dtype: float64


In [11]:
# IC Violations and Constraints

# check for duplicate rows
df = menu_page_clean
dups = df[df.duplicated(subset=['id'], keep=False)]
display(dups)

# FK and PK
# PK Verification
counts = df.groupby('id')['id'].nunique()
bad_id = counts[counts > 1].index
bad_df = df[df['id'].isin(bad_id)]
display(bad_df)

pk_violations = bad_df.groupby(['id', 'id']).size().to_dict()
pk_violations

# FK Verification
bad_fk_menu_page = df[~df["menu_id"].isin(menu["id"])]
display(bad_fk_menu_page)

,id,menu_id,page_number,image_id,full_height,full_width,uuid


,id,menu_id,page_number,image_id,full_height,full_width,uuid


,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99
...,...,...,...,...,...,...,...
66922,77417,35525,2.0,5194206,2942.0,1990.0,af4751c0-6212-0132-6245-58d385a7bbd0
66923,77418,35525,3.0,5194207,2837.0,2628.0,af604d50-6212-0132-b41d-58d385a7bbd0
66924,77419,35525,4.0,5194208,2852.0,2002.0,af7910c0-6212-0132-e227-58d385a7bbd0
66925,77420,35525,5.0,5194209,2835.0,1990.0,af92d3f0-6212-0132-6b8d-58d385a7bbd0


# Data Cleaning

In [1]:
# done in Open Refine